In [1]:
from DB.Chromadb import ChromaDB
from langchain.schema import Document
from glob import glob
import os
import pandas as pd


BASE_DIR = 'datas' # 데이터가 저장돼 있는 루트를 의미합니다.
csv_routes = glob(os.path.join(BASE_DIR, '*/*.csv'))
df = pd.DataFrame()
for route in csv_routes:
    d = pd.read_csv(route)
    df = pd.concat([df, d])
df = df.reset_index(drop=True)

df.sample(5)



,id,type,dir_route,original_content,file_name,page,investment,company_name,summary,image_route,table
490,67,paragraph,datas/SK하이닉스_한화투자증권(2024.11.27),당사는 산업에 대해 향후 1 년간 해당 업종의 수익률이 과거 수익률에 비해 양호한 ...,SK하이닉스_한화투자증권(2024.11.27).pdf,6,한화투자증권(2024.11.27),SK하이닉스,당사는 앞으로 1년간 해당 업종의 수익률이 좋을 것으로 예상되면 Positive(긍...,NaN,NaN
433,159,figure,datas/카카오뱅크_삼성증권(2024.11.27),NaN,카카오뱅크_삼성증권(2024.11.27).pdf,9,삼성증권(2024.11.27),카카오뱅크,"카카오뱅크의 2024년 DPS 예상치가 51.5%로 가장 높으며, 다른 은행들과 비...",datas/카카오뱅크_삼성증권(2024.11.27)/pag...,NaN
55,123,paragraph,datas/카카오뱅크_한국신용평가(2024.09.13),이 러한 보수적인 유동성 관리 정책과 충성도 높은 수신기반을 고려하면 향후에도 규제...,카카오뱅크_한국신용평가(2024.09.13).pdf,8,한국신용평가(2024.09.13),카카오뱅크,카카오뱅크는 현재까지 안정적인 유동성 관리와 충성도 높은 수신 기반을 갖추고 있으며...,NaN,NaN
675,3,paragraph,datas/크래프톤_삼성증권(2024.9.11),오동환 Senior Analyst dh1.oh@samsung.com,크래프톤_삼성증권(2024.9.11).pdf,1,삼성증권(2024.9.11),크래프톤,오동환 시니어 애널리스트는 '크래프톤'에 대한 분석 보고서를 발표했다. 이 보고서에...,NaN,NaN
569,45,figure,datas/크래프톤_유안타증권(2024.09.24),NaN,크래프톤_유안타증권(2024.09.24).pdf,4,유안타증권(2024.09.24),크래프톤,이 이미지는 2024년 9월 24일 기준으로 'BATTLEGROUNDS MOBILE...,datas/크래프톤_유안타증권(2024.09.24)/page...,NaN


In [2]:
# 모든 문서를 Document라는 형태로 리스트에 저장합니다.
# 정량평가를 위해서는 type이 paragraph인 문서들만 넣어야 하지만, 일단 전부 넣도록 합니다.
def create_documents(df):
    documents = []
    for _, row in df.iterrows():
        metadata = row.to_dict()
        text = metadata.pop("summary") 
        documents.append(Document(page_content=text, metadata=metadata))
    return documents

In [3]:
documents = create_documents(df[:200])

In [4]:

documents[:5]


[Document(metadata={'id': 5, 'type': 'paragraph', 'dir_route': 'datas/카카오뱅크_한국신용평가(2024.09.13)', 'original_content': '김경근 선임애널리스트 02-787-2383 kyungkeun.kim@kisrating.com', 'file_name': '카카오뱅크_한국신용평가(2024.09.13).pdf', 'page': 1, 'investment': '한국신용평가(2024.09.13)', 'company_name': '카카오뱅크', 'image_route': nan, 'table': nan}, page_content='김경근 선임애널리스트가 쓴 카카오뱅크 관련 한국신용평가 리포트 입니다.'),
 Document(metadata={'id': 6, 'type': 'paragraph', 'dir_route': 'datas/카카오뱅크_한국신용평가(2024.09.13)', 'original_content': '위 지원 실장 02-787-2279 geewon.wee@kisrating.com', 'file_name': '카카오뱅크_한국신용평가(2024.09.13).pdf', 'page': 1, 'investment': '한국신용평가(2024.09.13)', 'company_name': '카카오뱅크', 'image_route': nan, 'table': nan}, page_content='위 지원 실장, ***-***-****, g*********@*********.***으로 연락 가능한 한국신용평가에서 카카오뱅크에 대한 리포트를 발표함.'),
 Document(metadata={'id': 10, 'type': 'figure', 'dir_route': 'datas/카카오뱅크_한국신용평가

In [5]:
# add_docs를 통해 문서들을 저장합니다.

collection_name = 'chrdb.db'
DB = ChromaDB(BASE_DIR, collection_name)
DB.add_docs(documents)



In [6]:
# verify_db를 통해 db내의 데이터 수를 확인할 수 있고, db객체를 받을 수 있습니다.
db = DB.verify_db()


Total documents in collection 'chrdb.db': 200


In [7]:
# 위처럼 import하여 kiwi tokenizer을 사용하는 bm25와 dpr을 생성할 수 있습니다.
from retrievals import bm25, dpr
DPRretirever = dpr(db, topk = 3)
BM25Retriever = bm25(documents, topk = 3)


In [12]:
query = '네이버의 2024년 매출을 알려줘.'
DPRretirever.invoke(query)

[Document(metadata={'company_name': '네이버', 'dir_route': 'datas/네이버_SK증권(2024.11.18)', 'file_name': '네이버_SK증권(2024.11.18).pdf', 'id': 57, 'image_route': 'datas/네이버_SK증권(2024.11.18)/page_4_id_57_image.png', 'investment': 'SK증권(2024.11.18)', 'page': 4, 'table': '| 항목                     | 2022   | 2023   | 2024E  | 2025E  | 2026E  |\n|------------------------|--------|--------|--------|--------|--------|\n| 매출액                  | 8,220  | 9,671  | 10,621 | 11,524 | 12,571 |\n| 매출원가                | 0      | 0      | 0      | 0      | 0      |\n| 매출총이익              | 8,220  | 9,671  | 10,621 | 11,524 | 12,571 |\n| 매출총이익률            | 100.0  | 100.0  | 100.0  | 100.0  | 100.0  |\n| 판매비와 관리비 영업이익 | 6,915  | 8,182  | 8,652  | 9,156  | 9,869  |\n| 영업이익률              | 1,305  | 1,489  | 1,969  | 2,369  | 2,702  |\n| 비영업손익              | -221   | -7     | 393    | 123    | 205    |\n| 순금융손익              | -13    | -26    | 161    | 175    | 191    |\n| 외환관련손익         

In [13]:
BM25Retriever.invoke(query)

[Document(metadata={'id': 54, 'type': 'table', 'dir_route': 'datas/네이버_SK증권(2024.11.18)', 'original_content': nan, 'file_name': '네이버_SK증권(2024.11.18).pdf', 'page': 4, 'investment': 'SK증권(2024.11.18)', 'company_name': '네이버', 'image_route': 'datas/네이버_SK증권(2024.11.18)/page_4_id_54_image.png', 'table': '| 항목                     | 2022    | 2023    | 2024E   | 2025E   | 2026E   |\n|------------------------|---------|---------|---------|---------|---------|\n| **영업활동현금흐름**       |         |         |         |         |         |\n| 당기순이익(손실)          | 1,453   | 2,002   | 499     | 2,943   | 3,302   |\n| 비현금성항목목록          | 1,318   | 1,327   | 1,432   | 1,151   | 1,218   |\n| 유형자산감가상각비        | 519     | 528     | 640     | 738     | 733     |\n| 무형자산상각비            | 42      | 54      | 51      | 56      | 55      |\n| 기타                     | 757     | 745     | 741     | 357     | 430     |\n| **투자활동현금흐름**       |         |         |         |         |       